In [1]:
import sqlite3
import pandas as pd
import stringdist
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext sql

In [2]:
%sql sqlite:///results.db

## Autocorrect

In [24]:
%%sql renames <<
select alias.name, command.name 
from alias join command using (alias_id)
where alias.value = command.name

 * sqlite:///results.db
Done.
Returning data to local variable renames


In [58]:
import stringdist

autocorrects = []

for (alias,cmd) in renames:
    dist = stringdist.rdlevenshtein_norm(alias, cmd)
    if len(alias) > 2 and len(alias) == len(cmd) and dist > 0 and dist >= 0.4: # and dist < 0.5 and len(alias) == len(cmd):
        autocorrects.append((alias, cmd, dist))

autocorrects.sort(key=lambda tup: tup[2])
        
# TODO: count sl = ls and dc = cd etc.
        
print(len(autocorrects))
        
print(*autocorrects[:200], sep='\n')

9251
('eacms', 'emacs', 0.4)
('ecmas', 'emacs', 0.4)
('ecams', 'emacs', 0.4)
('emcsa', 'emacs', 0.4)
('mecas', 'emacs', 0.4)
('blaze', 'bazel', 0.4)
('clipg', 'gclip', 0.4)
('claro', 'clear', 0.4)
('paint', 'pinta', 0.4)
('caler', 'clear', 0.4)
('emcsa', 'emacs', 0.4)
('clrea', 'clear', 0.4)
('clrae', 'clear', 0.4)
('clare', 'clear', 0.4)
('ecams', 'emacs', 0.4)
('vlaut', 'vault', 0.4)
('ospkg', 'eopkg', 0.4)
('解压zip', 'unzip', 0.4)
('blaze', 'bazel', 0.4)
('blaze', 'bazel', 0.4)
('blaze', 'bazel', 0.4)
('ermac', 'emacs', 0.4)
('CMake', 'cmake', 0.4)
('blaze', 'bazel', 0.4)
('ysint', 'yinst', 0.4)
('prn_enable', 'cupsenable', 0.4)
('soncs', 'scons', 0.4)
('paint', 'pinta', 0.4)
('ermac', 'emacs', 0.4)
('caler', 'clear', 0.4)
('caler', 'clear', 0.4)
('blaze', 'bazel', 0.4)
('corgi', 'cargo', 0.4)
('clare', 'clear', 0.4)
('ckaer', 'clear', 0.4)
('mecas', 'emacs', 0.4)
('clrea', 'clear', 0.4)
('setgr', 'setfg', 0.4)
('list.icons', 'show.icons', 0.4)
('killagents', 'stopagents', 0.4)
('bla

In [ ]:
# Given a command, how many aliases exist for this command that would be considered autocorrecting
def autocorrect_count(command):
    # The distance measures for short commands cannot be
    # interpreted in the same way (as, for instance, ls and ll do indeed
    # have different semantics and are not considered misspellings)
    if len(command) < 3:
        return 0
    resultset = %sql select a.name as alias, c.name as command from alias a join command c on a.alias_id = c.alias_id where c.name = :command and length(a.name) = length(c.name) and a.name <> c.name and length(a.name) > 2
        
    import stringdist
    # calculate distance between alias and command
    distance_items = [stringdist.rdlevenshtein_norm(item[0], item[1]) for item in resultset]
    # filter for empirically determined threshold
    threshold = 0.33
    return len(list(filter(lambda item: item <= threshold, distance_items)))



cmds = [
'apt', 'apt-get', 'apt-cache', 'aptitude'

]

for cmd in cmds:
    count = autocorrect_count(cmd)
    totalR = %sql select count(*) from command where name = :cmd
    total = totalR[0][0]
    print(f'{cmd} {round(count*100.0/total,2)}')